In [399]:
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer, CountVectorizer
import glob
import os 
import pandas as pd
import datetime
from scipy.stats import iqr
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split

from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

spm = None
english_words = None
excepted_stock_names = []
limit = -1

In [400]:
import keras
from keras.layers import Dense, Dropout, RepeatVector, BatchNormalization, Convolution1D, Flatten, Lambda, Permute, MaxPooling1D, AlphaDropout
from keras.models import Sequential
from keras.utils import to_categorical
import keras.backend as K
from sklearn.model_selection import train_test_split
from keras.models import load_model
from tqdm import *

In [401]:
def stock_name_from_filename(filename):
    return filename.split('/')[-1].split('_')[0]

In [402]:
def stock_name_date_mapping(limit=-1):
    spm = {}
    for i, file in enumerate(glob.glob('filing_texts/*')[0:limit]):
        try:
            stock_name = stock_name_from_filename(file)
            file_data = open(file, 'r').read()[:200]
            time_data = file_data.split('<ACCEPTANCE-DATETIME>')[1].split('\\n')[0][:8]
            year = time_data[0:4]
            month = time_data[4:6]
            day = time_data[6:8]
            date_stamp = ("%s-%s-%s" %(year, month, day))
            spm[stock_name] = {'date': date_stamp}
        except:
            print("No data for ", stock_name)
            excepted_stock_names.append(stock_name)
            continue
    return spm

In [403]:
class PriceData:
    def __init__(self, stock_name):
        self.stock_name = stock_name
        try:
            self.price_data = pd.read_csv('prices/' + stock_name + '.csv')
        except:
            self.price_data = pd.DataFrame.from_dict({})

    def on_date(self, date, market_time = 'open'): 
        try:
            return float(self.price_data.loc[self.price_data['date'] == date][market_time])
        except: 
            return None

In [404]:
class FilenamesToStockNamesTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        return None
    
    def fit(self, X, y=None):
        return self
    
    def fit_transform(self, X, y=None):
        return self.fit(X, y).transform(X)
        
    def transform(self, X):
        output = []
        for filename in X:
            stock_name = self.__stock_name_from_filename(filename)
            output.append(stock_name)
        return output
    def __stock_name_from_filename(self, filename):
        return filename.split('/')[-1].split('_')[0]

In [405]:
## Used in y pipeline
class SpmToFileNamesTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        return None
    
    def fit(self, X, y=None):
        return self
    
    def fit_transform(self, X, y=None):
        return self.fit(X, y).transform(X)
    def transform(self, X):
        output = []
        for key in X.keys():
            date = X[key]['date']
            output.append(f'filing_texts/{key}_{date}')
        return output
    
class SpmToStockNamesTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        return None
    
    def fit(self, X, y=None):
        return self
    
    def fit_transform(self, X, y=None):
        return self.fit(X, y).transform(X)
        
    def transform(self, X):
        return X.keys()
    
class StockNamesToFileNamesTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        return None
    
    def fit(self, X, y=None):
        return self
    
    def fit_transform(self, X, y=None):
        return self.fit(X, y).transform(X)
        
    def transform(self, X):
        return [f'filing_texts/{stock_name}' for stock_name in X]
    
class MapStockNamesToDatesTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, start_int, end_int):
        self.start_int = start_int
        self.end_int = end_int
        self.range = (range(start_int, end_int))
    
    def fit(self, X, y=None):
        return self
    
    def fit_transform(self, X, y=None):
        return self.fit(X, y).transform(X)
        
    def transform(self, X):
        output = {}
        for i, stock_name in enumerate(X):
            try:
                date = spm[stock_name]['date']
            except:
                continue # Ignore stocks which don't have a date
            dates = []
            for delta in self.range:
                date_delta = datetime.timedelta(days=delta)
                date_string = datetime.datetime.strptime(date, '%Y-%m-%d').date()
                dates.append(str(date_string + date_delta))
            output[stock_name] = dates
        return output
class StockNameDatesMapToPricesListTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        return None
    
    def fit(self, X, y=None):
        return self
    
    def fit_transform(self, X, y=None):
        return self.fit(X, y).transform(X)
        
    def transform(self, X):
        output = []
        for stock_name in X.keys():
            prices = []
            for date in X[stock_name]:
                prices.append(PriceData(stock_name).on_date(date, 'close'))
            output.append(prices)
        return np.array(output)
class LabelsTransform(BaseEstimator, TransformerMixin):
    # Returns the interquartile-range and median.
    def __init__(self):
        return None
        
    def fit(self, X, y=None):
        return self
    
    def fit_transform(self, X, y=None):
        self.fit(X, y).transform(X)
        
    def transform(self, X):
        # ldcom = last_day_change_over_median
        ldcom = []
        for prices in X:
            this_median = np.median(prices[0:-3])
            ldcom.append(((prices[-1]-this_median)/this_median))
        return np.array(ldcom)
            
class StatisticalMeasuresTransformer(BaseEstimator, TransformerMixin):
    # Returns the interquartile-range and median.
    def __init__(self):
        return None
        
    def fit(self, X, y=None):
        # ldcom = last_day_change_over_median
        
        output = []
        self.iqr_var = []
        self.median = []
        for prices in X:
            this_iqr = iqr(prices[0:-3])
            this_median = np.median(prices[0:-3])
            self.iqr_var.append(this_iqr)
            self.median.append(this_median)
        return self
    
    def fit_transform(self, X, y=None):
        return self.fit(X, y).transform(X)
        
    def transform(self, X):
        X_output = []
        for i, prices in enumerate(X):
            stats   = []
            iqr_var = self.iqr_var[i] or iqr(prices)
            median  = self.median[i]  or np.median(prices)
            
            stats.append(iqr_var)
            stats.append(median)
            
            X_output.append(stats)
        return np.array(X_output)
class SparseToArray(BaseEstimator, TransformerMixin):
    def __init__(self):
        return None
        
    def fit(self, X, y=None):
        return self
    
    def fit_transform(self, X, y=None):
        return self.fit(X, y).transform(X)
        
    def transform(self, X):
        return np.array(X.toarray()) #[ar.toarray() for ar in X]
    
class ReadFiles(BaseEstimator, TransformerMixin): 
    def __init__(self):
        return None
        
    def fit(self, X, y=None):
        return self
    
    def fit_transform(self, X, y=None):
        return self.fit(X, y).transform(X)
        
    def transform(self, X):
        return (open(filename, 'r').read() for filename in tqdm(X))
            

In [406]:
def get_filenames(limit = -1):
    filenames = []
    directory_files = glob.glob('filing_texts/*')
    excepted_files = [('filing_texts/' + sn) for sn in excepted_stock_names]
    filenames_with_data = [x for x in directory_files[:limit] if x not in excepted_files]
    for filename in filenames_with_data:
        filenames.append(filename)
    return filenames

In [407]:
spm = stock_name_date_mapping(limit)

filenames = get_filenames(limit)

In [442]:
# Used to build other pipelines

prices_pipeline = Pipeline([
    ('spm_to_filenames', SpmToFileNamesTransformer()),
    ('filenames_to_stock_names', FilenamesToStockNamesTransformer()),
    ('stock_names_to_dates', MapStockNamesToDatesTransformer(-3, 2)),
    ('dates_to_prices_transformer', StockNameDatesMapToPricesListTransformer()),
    ('imputer', SimpleImputer())
])

# Used as the final y values 
labels_pipeline = Pipeline([
    ('prices_pipeline', prices_pipeline),
    ('labels_transform', LabelsTransform())
])

# Used for training and test set features
stock_stats = Pipeline([
    ('prices_pipeline', prices_pipeline),
    ('stats_transform', StatisticalMeasuresTransformer())
])

# Used for training and test set features 
# ('stock_names_to_file_names', StockNamesToFileNamesTransformer()),
text_word_counts = Pipeline([
    ('spm_to_file_names', SpmToFileNamesTransformer()),
    ('read_files', ReadFiles()),
    ('vect', TfidfVectorizer(
                token_pattern=r"[a-zA-Z]+", 
                min_df = 0.10,
                max_df = 0.80,
                stop_words = 'english',
                max_features=9000,
                ngram_range=(1, 1))),
    ('sparse_to_array', SparseToArray()),
    ('std_scaler', StandardScaler()),
])


In [443]:
X = text_word_counts.fit_transform(spm)
y = labels_pipeline.fit_transform(spm)
print(len(X))
print(len(y))






  0%|          | 0/192 [00:00<?, ?it/s]




  1%|          | 2/192 [00:00<00:19,  9.60it/s]




  6%|▌         | 11/192 [00:00<00:05, 32.84it/s]




  7%|▋         | 14/192 [00:01<00:23,  7.57it/s]




 10%|█         | 20/192 [00:02<00:17,  9.69it/s]




 13%|█▎        | 25/192 [00:02<00:15, 10.75it/s]




 17%|█▋        | 32/192 [00:02<00:12, 13.16it/s]




 19%|█▉        | 36/192 [00:02<00:12, 12.10it/s]




 21%|██        | 40/192 [00:03<00:11, 12.68it/s]




 22%|██▏       | 43/192 [00:03<00:11, 12.71it/s]




 27%|██▋       | 51/192 [00:03<00:09, 14.61it/s]




 29%|██▉       | 56/192 [00:03<00:08, 15.58it/s]




 32%|███▏      | 61/192 [00:03<00:08, 16.27it/s]




 34%|███▍      | 65/192 [00:03<00:07, 16.57it/s]




 36%|███▌      | 69/192 [00:06<00:11, 10.86it/s]




 40%|████      | 77/192 [00:06<00:10, 11.27it/s]




 43%|████▎     | 83/192 [00:07<00:09, 11.85it/s]




 46%|████▌     | 88/192 [00:07<00:08, 11.90it/s]




 49%|████▉     | 95/192 [00:07<00:07, 12.66it/s]




192


TypeError: object of type 'NoneType' has no len()

In [348]:
print(X.shape)
print(y.shape)

(192, 3700)
(192,)


In [380]:
X_train, X_test, y_train_continuous, y_test_continuous = train_test_split(X, y, test_size=0.2, random_state=42)

In [384]:
def bool_arr(arr):
    y_bool = []
    for num in arr:
        if num >= 0.1:
            y_bool.append(1)
        else:
            y_bool.append(0)
    return np.array(y_bool)

In [531]:
from sklearn.feature_selection import SelectFpr
from sklearn.model_selection import GridSearchCV, cross_val_score

train_pipeline = Pipeline([
    ('reduce_false_pos', SelectFpr(alpha=0.33)),
    ('svc', GridSearchCV(estimator=SVC(gamma='auto'), param_grid=dict(C=np.logspace(-1,3)), n_jobs=1))
])

In [532]:
y_train = bool_arr(y_train_continuous)
y_test = bool_arr(y_test_continuous)

In [533]:
train_pipeline.fit(X_train, y_train)

/Users/Piper/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:734: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Pipeline(memory=None,
     steps=[('reduce_false_pos', SelectFpr(alpha=0.33, score_func=<function f_classif at 0x1a2e6e78c8>)), ('svc', GridSearchCV(cv=None, error_score='raise-deprecating',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kerne...   pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0))])

In [534]:
train_pipeline.score(X_test, y_test)

0.58974358974358976

In [539]:
# train_pipeline.predict(X_test)
print(economic_score(X_test, y_test_continuous, train_pipeline))

[22.899769244214269, 43.727389956429228]


In [441]:
from sklearn.svm import SVC

clf = SVC(C=100)
# clf = RandomForestRegressor(n_estimators=200, max_depth=3, verbose=1, n_jobs=2)
clf.fit(lil_x, y_train)

print(clf.score(select_fpr.transform(X_train), y_train))
print(clf.score(select_fpr.transform(X_test), y_test))


0.941176470588
0.589743589744


/Users/Piper/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:194: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [536]:
def economic_score(feat, labels, clf):
    total = np.dot(m1.predict(feat).reshape(feat.shape[0],), labels)
    buy_all = np.dot(np.ones(feat.shape[0]), labels)
    av = total/len(feat)
    buy_all_av = buy_all/len(feat)
    return [av, buy_all_av]

In [391]:
print(economic_score(X_train, y_train_continuous, m1))
print(economic_score(X_test, y_test_continuous, m1))
print(economic_score(X_test, y_test_continuous, clf))

[90.120230661697605, 185.02618425699143]
[22.899769244214269, 43.727389956429228]
[22.899769244214269, 43.727389956429228]


In [331]:
# # Factors to look into including: 
# "BookValue" = (Total Assets - Total Liabilities) / Number of shares outstanding
# "MarketCap" = Market price per share * number of shares 
# "DividendYield" = Dividend / Market price per share 
# "EarningsPerShare" 
# "PERatio2" = Market price per share / earning per share 
# "priceBook" = Market price per share / ((Total Assets - Total Liabilities) / Number of shares outstanding)
# "PriceSales" = MarketCap / Revenue 
# "Ask"


In [332]:
def bool_arr(arr):
    y_bool = []
    for num in arr:
        if num >= 0.1:
            y_bool.append(1)
        else:
            y_bool.append(0)
    return np.array(y_bool)
    

In [335]:
from keras import regularizers

shape = X_train.shape[1]

m1 = Sequential([
    BatchNormalization(input_shape=(shape,)),
    Dense(35, activation='relu', kernel_regularizer=regularizers.l1(0.01)),
    Dropout(0.5),
    BatchNormalization(),
    Dense(1, activation='sigmoid')   
])

m1.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
m1

In [336]:
# sched = [[0.0001, 1000]]
sched = [[0.0001, 2], [0.001, 20], [0.01, 2], [0.1, 2], [0.5, 1], [0.1, 5], [0.01, 20], [0.001, 40], [0.0001, 80], [0.00005, 120]]

for i in range(1):
    for info in sched:
        lr, epochs = info
        m1.optimizer.lr = lr
        m1.fit(np.array(X_train), np.array(y_train_bool), epochs=epochs,  batch_size=64, validation_data=(np.array(X_test), np.array(y_test_bool)))

Train on 153 samples, validate on 39 samples
Epoch 1/2
153/153 [==============================] - 1s 4ms/step - loss: 26.7506 - acc: 0.4641 - val_loss: 26.6236 - val_acc: 0.4103
Epoch 2/2
153/153 [==============================] - 0s 390us/step - loss: 26.4335 - acc: 0.5229 - val_loss: 26.3072 - val_acc: 0.4359
Train on 153 samples, validate on 39 samples
Epoch 1/20
153/153 [==============================] - 0s 307us/step - loss: 26.0977 - acc: 0.4967 - val_loss: 26.0022 - val_acc: 0.4359
Epoch 2/20
153/153 [==============================] - 0s 402us/step - loss: 25.7170 - acc: 0.5556 - val_loss: 25.7036 - val_acc: 0.4359
Epoch 3/20
153/153 [==============================] - 0s 385us/step - loss: 25.4699 - acc: 0.6013 - val_loss: 25.4087 - val_acc: 0.4615
Epoch 4/20
153/153 [==============================] - 0s 474us/step - loss: 25.2408 - acc: 0.5359 - val_loss: 25.1165 - val_acc: 0.4615
Epoch 5/20
153/153 [==============================] - 0s 384us/step - loss: 24.9465 - acc: 0.5163 

153/153 [==============================] - 0s 372us/step - loss: 12.7932 - acc: 0.8627 - val_loss: 13.0789 - val_acc: 0.4872
Epoch 7/40
153/153 [==============================] - 0s 325us/step - loss: 12.6430 - acc: 0.8301 - val_loss: 12.8934 - val_acc: 0.4872
Epoch 8/40
153/153 [==============================] - 0s 427us/step - loss: 12.5073 - acc: 0.8301 - val_loss: 12.7098 - val_acc: 0.4872
Epoch 9/40
153/153 [==============================] - 0s 514us/step - loss: 12.2785 - acc: 0.8693 - val_loss: 12.5347 - val_acc: 0.5385
Epoch 10/40
153/153 [==============================] - 0s 537us/step - loss: 12.0889 - acc: 0.8497 - val_loss: 12.3591 - val_acc: 0.5128
Epoch 11/40
153/153 [==============================] - 0s 576us/step - loss: 11.9075 - acc: 0.8824 - val_loss: 12.1828 - val_acc: 0.5385
Epoch 12/40
153/153 [==============================] - 0s 434us/step - loss: 11.7418 - acc: 0.8627 - val_loss: 12.0067 - val_acc: 0.4872
Epoch 13/40
153/153 [==============================] - 0

153/153 [==============================] - 0s 451us/step - loss: 4.7967 - acc: 0.9542 - val_loss: 5.3325 - val_acc: 0.5385
Epoch 27/80
153/153 [==============================] - 0s 555us/step - loss: 4.7408 - acc: 0.9608 - val_loss: 5.2635 - val_acc: 0.5128
Epoch 28/80
153/153 [==============================] - 0s 566us/step - loss: 4.6578 - acc: 0.9281 - val_loss: 5.2027 - val_acc: 0.5128
Epoch 29/80
153/153 [==============================] - 0s 493us/step - loss: 4.6073 - acc: 0.9346 - val_loss: 5.1451 - val_acc: 0.5128
Epoch 30/80
153/153 [==============================] - 0s 346us/step - loss: 4.5278 - acc: 0.9281 - val_loss: 5.0838 - val_acc: 0.5128
Epoch 31/80
153/153 [==============================] - 0s 349us/step - loss: 4.4376 - acc: 0.9477 - val_loss: 5.0089 - val_acc: 0.5128
Epoch 32/80
153/153 [==============================] - 0s 712us/step - loss: 4.3862 - acc: 0.9412 - val_loss: 4.9335 - val_acc: 0.5385
Epoch 33/80
153/153 [==============================] - 0s 456us/ste

153/153 [==============================] - 0s 368us/step - loss: 2.2315 - acc: 0.9608 - val_loss: 2.8661 - val_acc: 0.4872
Epoch 7/120
153/153 [==============================] - 0s 347us/step - loss: 2.2179 - acc: 0.9477 - val_loss: 2.8350 - val_acc: 0.5128
Epoch 8/120
153/153 [==============================] - 0s 413us/step - loss: 2.1951 - acc: 0.9608 - val_loss: 2.7868 - val_acc: 0.5385
Epoch 9/120
153/153 [==============================] - 0s 436us/step - loss: 2.1514 - acc: 0.9542 - val_loss: 2.7604 - val_acc: 0.5385
Epoch 10/120
153/153 [==============================] - 0s 384us/step - loss: 2.1703 - acc: 0.9477 - val_loss: 2.7639 - val_acc: 0.4615
Epoch 11/120
153/153 [==============================] - 0s 426us/step - loss: 2.1338 - acc: 0.9477 - val_loss: 2.7395 - val_acc: 0.4872
Epoch 12/120
153/153 [==============================] - 0s 457us/step - loss: 2.1034 - acc: 0.9608 - val_loss: 2.7105 - val_acc: 0.4615
Epoch 13/120
153/153 [==============================] - 0s 429us

153/153 [==============================] - 0s 310us/step - loss: 1.2227 - acc: 0.9477 - val_loss: 1.9350 - val_acc: 0.5641
Epoch 67/120
153/153 [==============================] - 0s 277us/step - loss: 1.2440 - acc: 0.9281 - val_loss: 1.9395 - val_acc: 0.5897
Epoch 68/120
153/153 [==============================] - 0s 319us/step - loss: 1.2629 - acc: 0.9216 - val_loss: 1.9601 - val_acc: 0.5897
Epoch 69/120
153/153 [==============================] - 0s 319us/step - loss: 1.2157 - acc: 0.9673 - val_loss: 1.9597 - val_acc: 0.5897
Epoch 70/120
153/153 [==============================] - 0s 381us/step - loss: 1.2407 - acc: 0.9608 - val_loss: 1.9808 - val_acc: 0.5128
Epoch 71/120
153/153 [==============================] - 0s 334us/step - loss: 1.2237 - acc: 0.9739 - val_loss: 1.9569 - val_acc: 0.5385
Epoch 72/120
153/153 [==============================] - 0s 353us/step - loss: 1.1995 - acc: 0.9673 - val_loss: 1.9513 - val_acc: 0.5385
Epoch 73/120
153/153 [==============================] - 0s 34

In [337]:
economic_score(X_test, y_test, m1)

22.899769244214269

In [344]:
[float(y) for y in y_test]

[-0.341781988102027,
 6.622165640631012,
 -0.436573573614911,
 0.4068140965980122,
 -0.3204552325076958,
 5.615243605155948,
 0.2844426590633416,
 -0.7908905068043499,
 -0.4251771038750514,
 0.736151415826052,
 26.784023141654984,
 219.9040054863566,
 -0.2094495480575452,
 308.3766944652674,
 -0.9440548025535945,
 -0.4406530114550679,
 0.05152572017007015,
 -0.8919876257041239,
 -0.10538475171729224,
 0.29631114248398777,
 -0.685671118315194,
 573.2031449275363,
 -0.30100250171132575,
 -0.9266159679220135,
 391.393948697633,
 -0.9648233763443692,
 -0.5546952403764974,
 -0.7321554236118895,
 -0.6012213774010581,
 -0.6388810878406338,
 -0.13477359529591648,
 1.6420390104027747,
 181.17104851761937,
 0.2695291234482818,
 -0.4423768500638972,
 -0.8860388391498305,
 -0.46980137033134683,
 0.3490179078404973,
 0.506567635807774]

In [ ]:
# vocab = list(text_word_counts.steps[2][1].vocabulary_.keys())
# import operator
# iv_dict = [[vocab[i],-float(f)] for i,f in enumerate(clf.coef_)]
# most_important_terms = sorted(iv_dict, key=operator.itemgetter(1))[0:100]
# print(most_important_terms)
# most_important_vocab = dict(most_important_terms).keys()

In [ ]:
# from joblib import Memory
# %mkdir cachedir
# location = './cachedir'
# memory = Memory(location, verbose=0)
# stock_name_date_mapping = memory.cache(stock_name_date_mapping)

In [ ]:
# # print(text_word_counts.steps[1][1].vocabulary_)
# def get_excepted_stock_names(limit=-1):
#     esn = {}
#     for i, file in tqdm(enumerate(glob.glob('filing_texts/*')[0:limit])):
#         stock_name = stock_name_from_filename(file)
#         file_data = open(file, 'r').read()
#         try:
#             with open(file) as open_file:
#                 file_data = [next(open_file) for x in range(3)]
#             file_data = ''.join(file_data)
#             time_data = file_data.split("\n")[2][5:14]
#         except:
#             esn[i] = stock_name
#             continue
#     return esn

# def get_x_mask(X, y, filenames):
#     no_date_indices = []
#     if X.shape[0] != len(y):
#         stock_names = FilenamesToStockNamesTransformer().transform(filenames)
#         for i, stock_name in enumerate(stock_names): 
#             try:
#                 date = spm[stock_name]['date']
#             except:
#                 no_date_indices.append(i)
#         mask = np.ones(X.shape[0], dtype=bool)
#         mask[no_date_indices] = False
#         X_mask = X[mask]
#     else:
#         X_mask = X
#     return X_mask